<a href="https://colab.research.google.com/github/davidwhogg/FlatPriorFlatLikelihood/blob/main/notebooks/To_Gaby_From_Hogg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# For Gaby: flat LF, flat priors, and yet peaked posterior

## Authors:
- **David W. Hogg** (NYU) (MPIA)

## Projects:
- Show that even in the simplest situation: Everything linear, everything Gaussian, and flat priors, the Bayesian posterior is peaked when the likelihood function is not.
  - That is, even integrating a flat LF over a flat prior can give you a spurious peak. It's sweet! The fundamental reason is that in more than a few dimensions, any misalignment between the model degeneracy and the box edges will give you structure.

## Bugs / To-do items:
- Show that as the truth (`CENTER`) moves, the peak in the LF moves accordingly.
- Explore the effect of dimensionality of the space.
- Explore the effect of the dimensionality of the degeneracy.
- Show plots that are aligned with the degeneracy.
- Show that the problem is easy to diagnose by changing the limits of the prior.
- Switch to a latin hypercube, maybe?


In [ ]:
import numpy as np
import pylab as plt

In [ ]:
# hyperparameters / choices
P = 6 # dimensionality of the space
Q = 2 # dimensionality of the null space (exact degeneracy space)
M = 12 # number of samplings to do
N = 2 ** (13 + P // 3) # number of samples per sampling
print(P, Q, M, N)

In [ ]:
# make some random numbers to define our Universe
rng = np.random.default_rng(17)
CENTER = 10. * rng.uniform(size=P)
VECS = rng.normal(size=(P, P))

In [ ]:
# now orthogonalize to make a randomly oriented orthonormal coordinate system
UNITVECS = np.zeros((P, P))
for i in range(P):
    veci = 1. * VECS[i]
    for j in range(i):
        uvecj = UNITVECS[j]
        veci -= (veci @ uvecj) * uvecj
    UNITVECS[i] = veci / np.linalg.norm(veci)

In [ ]:
# Make a LLF with an exact M-dimensional degeneracy
def log_likelihood(points):
    """
    ## Bugs:
    - Only can take lists of points, not a single point.
    - Relies on global variables.
    """
    deltas = (UNITVECS[Q:] @ (points.T - CENTER[:, None])).T
    return 10.0 - 0.5 * np.sum(deltas * deltas, axis=1)

In [ ]:
# Choose a plot center that is near to a high-likelihood point, but integer
plot_center = np.round(CENTER + np.random.normal(size=CENTER.shape))
axis_labels = [f"\\theta_{{{i+1}}}" for i in range(P)]
print(plot_center, axis_labels)

In [ ]:
# Make MN samplings in a cube
halfsize = 5.0
points = 2. * halfsize * rng.uniform(size=(M * N, P)) - halfsize + plot_center[None, :]
llfs = log_likelihood(points)
print(points.shape, llfs.shape)

In [ ]:
FIGSIZE = 3. # units?
nx = np.ceil(np.sqrt(P)).astype(int)
ny = np.ceil(P / nx).astype(int)
fig, axs = plt.subplots(ny, nx, figsize=(FIGSIZE * nx, FIGSIZE * ny), sharey=True)
axs = axs.flatten()
for i in range(P):
    ax = axs[i]
    # ax.axvline(CENTER[i], lw=1, color="k", alpha=0.5)
    foo, bar = np.histogram(points[:, i], weights=np.exp(llfs[:]), density=True)
    xlabel = f"$p({axis_labels[i]}|Y)$"
    ax.stairs(foo, bar, color="k", alpha=1.0, fill=False, label=xlabel)
    for m in range(M):
        foo, bar = np.histogram(points[m::M, i], weights=np.exp(llfs[m::M]), density=True)
        ax.stairs(foo, bar, color="k", alpha=0.25, fill=False)
    # ax.legend(loc=8)
    ax.text(0.05, 0.95, f"${axis_labels[i]}$", transform=ax.transAxes,
            ha="left", va="top")
    ax.set_xlim(plot_center[i] - halfsize, plot_center[i] + halfsize)
for j in range(i+1, len(axs)):
    axs[j].set_visible(False)

In [ ]:
assert False

In [ ]:
Nplot = 2 ** 11
vmax = np.max(llfs)
vmin = vmax - 5.0
fig, axs = plt.subplots(P-1, P-1, figsize=(FIGSIZE * (P - 1), FIGSIZE * (P - 1)))
print(axs.shape)
for i in range(1, P):
    for j in range(i):
        ax = axs[i - 1, j]
        ax.scatter(points[:Nplot, j], points[:Nplot, i], s=4, c=llfs[:Nplot],
                   vmin=vmin, vmax=vmax, cmap="viridis_r", alpha=0.75)
        if i == (P - 1): ax.set_xlabel(axis_labels[j])
        if j == 0: ax.set_ylabel(axis_labels[i])
    for j in range(i, P - 1):
        axs[i - 1, j].set_visible(False)